In [ ]:
! pip install pandas
! pip install matplotlib
! pip install gensim

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import ast



In [19]:
df = pd.read_csv("../../processedDataset/preprocessedPostData.csv")

df.head()

,monthSin,monthCos,daySin,dayCos,hourSin,hourCos,weekdaySin,weekdayCos,hashtagVectors,nLikes,nShares,nFollowers,nComments,nViews,nAccountTotalLikes,postYear
0,-0.5,0.866025,0.988468,0.151428,-0.965926,-0.258819,-0.974928,-0.222521,[-0.06394912 -0.09934936 0.05018172 0.209499...,7.593374,1.386294,13.815512,2.944439,9.457279,15.363073,1.0
1,-0.5,0.866025,0.848644,0.528964,-0.258819,0.965926,0.433884,-0.900969,[-0.07862883 -0.10937244 0.03460374 0.202062...,8.905580,2.302585,13.815512,3.951244,10.874285,15.363073,1.0
2,-0.5,0.866025,0.724793,0.688967,-0.258819,-0.965926,0.974928,-0.222521,[-0.10907231 -0.05049138 0.07886021 0.172413...,8.439015,2.484907,13.815512,3.332205,10.537442,15.363073,1.0
3,-0.5,0.866025,0.571268,0.820763,-0.258819,0.965926,0.781831,0.623490,[-4.42794785e-02 -8.95778090e-02 4.70666662e-...,8.978660,1.945910,13.815512,3.218876,10.843514,15.363073,1.0
4,-0.5,0.866025,0.571268,0.820763,-0.258819,-0.965926,0.781831,0.623490,[-1.02420643e-01 -5.03370464e-02 7.44658336e-...,8.080237,2.302585,13.815512,2.708050,10.114599,15.363073,1.0


In [22]:
from ast import literal_eval

df['vectors'] = df['hashtagVectors'].apply(lambda x: np.array(literal_eval(x)))
# X = np.array([df['hashtagVectors']])
X = np.vstack(df['vectors'].values)
y = np.array(df['nLikes'].values)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


SyntaxError: invalid syntax (<unknown>, line 1)

In [13]:
from sklearn.linear_model import LogisticRegression

# Initialize the model
model = LogisticRegression(max_iter=1000)  # Increased max_iter to ensure convergence


In [14]:
model.fit(X_train, y_train)

ValueError: could not convert string to float: '[-6.23098947e-02 -7.48129711e-02  5.35395965e-02  1.92152575e-01\n  1.02765433e-01 -8.03307965e-02  2.37409875e-01  1.42219648e-01\n -2.85184919e-03 -8.96548554e-02 -3.12929042e-02  2.48000659e-02\n -1.61998361e-01 -1.13968536e-01  9.01136473e-02 -1.46625355e-01\n  6.49278471e-03  3.73223461e-02 -5.19672371e-02  2.32858807e-01\n  1.82414874e-01 -1.47371560e-01 -7.09674284e-02  2.45485857e-01\n -6.75048977e-02 -2.30783433e-01 -1.41616136e-01  7.19076246e-02\n -1.53688699e-01  8.83396193e-02  3.11371423e-02 -4.98276316e-02\n -8.51283818e-02  6.76052645e-02 -1.69197530e-01 -1.35109976e-01\n  3.71699743e-02 -1.41257033e-01  1.87388927e-01  1.01183355e-02\n  3.06853689e-02 -3.53892654e-01 -6.09709360e-02 -1.23922072e-01\n  2.50658691e-01  9.89419073e-02  6.81984723e-02  5.80841377e-02\n -2.15748828e-02 -9.69468206e-02  2.44854018e-01 -3.09965223e-01\n -1.57866493e-01 -2.85762131e-01 -1.55225154e-02  2.97067314e-01\n -8.54563937e-02  3.11399996